<a href="https://colab.research.google.com/github/baiascience/imdb/blob/main/IMDB_WebScraping_SQL_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


---

<p align=left>
<img src="https://github.com/baiascience/baia_science_main/blob/main/baia-science_black.png?raw=true" width="20%"></p>
by: Matheus Baia

---



# P1 - WebScraping do IMDB com SQL e Python 

<p align=center>
<img src="https://media.vlpt.us/images/miscaminos/post/4832cfc8-d693-4db9-8666-235147811ca4/crawler.png" width="60%"></p>


O processo de web scraping ou em bom português, raspagem de dados, é um processo de extração de dados de sites da web, colocando-os em formato estruturado por meio de manipulação para que posteriormente se realize variadas análises. Este processo pode ser útil para coleta de informações em redes sociais, sites governamentais ou qualquer informação que seja disponibilizada online. O processo permite a realização de análises a respeito destes dados, podendo ser uma poderosa arma de automatização de processos. Uma vez que, alguns departamentos empresariais precisam checar status de informações online e periodicamente, com o objetivo de tomar decisões data-driven. 

Infelizmente, e na maioria das vezes,  temos seres humanos realizando este processo manualmente. Que tal então realizar isto automaticamente? Neste artigo você será apresentado ao poder do Python, SQL e Power BI juntos para solucionar um desafio similar ao encontrado no mercado. Vamos realizar uma raspagem de dados em um site que publica a receita diária das bilheterias do cinema americano.

Este projeto tem como objetivo não só resgatar esses dados, mas manipulá-los, armazená-los em um banco de dados e divulgá-los utilizando uma ferramenta de visualização de dados, o Power Bi. Tudo isto em apenas um projeto.


<br>

<br>
Ao final deste projeto você será capaz de:

*   Entender como um processo de ** WebScraping** funciona com Python (Beautiful Soup)
*   Conhecer as típicas práticas de limpeza e manipulação de dados com **Python (Pandas)**
*   Conhecer as bibliotecas que viabilizam a exportação de dados para um banco de dados relacional por meio de **SQL , Python e o MS SQL Server**
*   Entender como realizar carregamento incremental no banco de dados e automatizações do processo, isto porque a cada dia, novos dados são divulgados e sempre queremos nosso banco devidamente atualizado.
*   Saber conectar o **Power BI** ao **MS SQL Server**

<br>

<br>

<p align=center>
<img src="https://www.diariodocentrodomundo.com.br/nao-assista-antes-de-consultar-como-o-imdb-se-tornou-o-site-sobre-cinema-mais-influente-do-mundo/imdb/" width="60%"></p>


<br>

## 1. Importação de bibliotecas e Captura de html

Primeiramente iremos realizar a importação do primeiro conjunto de bibliotecas Python úteis para o processo de WebScraping.

Observações:
- Comentários mais técnicos serão escritos em comentário Python como `#Comentário qualquer `
- Irei apresentando algumas linhas de código de forma desmembrada para entendermos como funciona a captura de página HTML e os passos básicos. Enquanto isso, serão apresentadas algumas funções que serão utilizadas de forma encadeada na solução final, caso o contrário, teriamos uma enorme repetição de linhas.


Iniciamos estabelecendo uma conexão de sucesso com o site em questão. A biblioteca urllib realizará a conexão com o site boxoffice do IMDB e a leitura da página HTML. Este [link](https://www.boxofficemojo.com/date/2021-02-11/)  mostra a página web que iremos extrair os dados. Para cada data há uma página web diferente.



In [1]:
import urllib.request as urllib_request
import time
import pandas as pd
import bs4
from datetime import date
from datetime import timedelta
from urllib.request import Request, urlopen

In [2]:
#Configuração do navegador web. Aqui simulamos o uso do navegador Mozilla, utilizando Windows 10. O HTML pode mudar sua apresentação conforme o navegador e o dispositivo se alteram. 

agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
headers = {'User-Agent': agente}

In [3]:
#Indicamos o endereço que gostariamos de regastar os dados, neste caso, o site boxofficemojo contém informações diárias sobre a receita dos cinemas americanos.

url = "https://www.boxofficemojo.com/date/2021-11-21/"

# Carregamento do agente que será usado na requisição
headers = {'User-Agent': agente}

# Efetuando a requisição e abrindo a página. É aqui que descobrimos se uma página não foi encontrada, quando isto ocorre um código HTML 404 é a resposta, a página web pode estar offline. 
# Quando temos sucesso na solicitação o código HTML 200 é devolvido. 
req = Request(url, headers = headers)
response = urlopen(req)


In [4]:
html = response.read() #Lendo o html

In [47]:
html[0:1000] #0:1000] limita a exibição a 1000 caracteres.

#Apresenta-se a página html pura, observe a extrema dificuldade que temos em ler qualquer dado. Isto ocorre porque diversas linguagem e protocolos estão impressos nesses dados. Javacript, CSS, HTML e suas tags. 
#Precisamos limpar os dados.

b'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);\n\n\n    var ue_err_chan = \'jserr-rw\';\n(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;


### Beautiful Soup - Biblioteca para limpeza dos dados

A biblioteca Beautiful Soup possui funções que permitirão a extração de dados a partir da identificação de algumas tags/regras do HTML. A partir de mera observação nota-se que os dados que gostariamos de obter estão entre tags <td>, que são típicas tags do formato tabela. O beautiful soup nos ajudará a extrair estes dados e deixar todo o restante de lado. 

<p align=center>
<img src="https://www.crummy.com/software/BeautifulSoup/bs4/doc/_images/6.1.jpg" width="30%"></p>





In [6]:
# Aplicação da primeira função que elimina a poluição do texto html.
soup = bs4.BeautifulSoup(html, 'html.parser')

In [7]:
soup;
# Para evitar um output longo irei suprimí-lo e incluir um trecho do output do objeto soup. Caso queira executar o bloco, recomendo abrir o código no google colab.



**Resumo de output do beautiful soup puro:**

<p align=center>
<img src="https://github.com/baiascience/imdb/blob/main/soup.png?raw=true" width="100%"></p>


==================================================================

In [8]:
# O método prettify do beautiful soup inclui a identação, o que facilita a leitura dos dados

# Para evitar um output longo irei suprimí-lo e incluir um trecho do output com o processo prettify(). Caso queira executar o bloco, recomendo abrir o código no google colab.

#print(soup.prettify())


**Resumo de output do soup.prettify():**

<p align=center>
<img src="https://github.com/baiascience/imdb/blob/main/prettify.png?raw=true" width="100%"></p>


==================================================================

In [46]:
#Como citado anteriormente, a função findAll irá capturar somente os dados com tag <td>
soup.findAll('td')[0:20] #[0:20] apresenta apenas as primeiras 20 linhas <td>. Esta ainda não é a tabela desejada.


[<td class="a-text-right mojo-header-column mojo-truncate mojo-field-type-rank mojo-sort-column">1</td>,
 <td class="a-text-right mojo-field-type-positive_integer">1</td>,
 <td class="a-text-left mojo-field-type-release mojo-cell-wide"><a class="a-link-normal" href="/release/rl319391233/?ref_=bo_da_table_1">Ghostbusters: Afterlife</a></td>,
 <td class="a-text-right mojo-field-type-money mojo-estimatable">$10,918,188</td>,
 <td class="a-text-right mojo-number-negative mojo-number-delta mojo-field-type-percent_delta mojo-estimatable">-33.6%</td>,
 <td class="a-text-right mojo-field-type-percent_delta mojo-estimatable">-</td>,
 <td class="a-text-right mojo-field-type-positive_integer mojo-estimatable">4,315</td>,
 <td class="a-text-right mojo-field-type-money mojo-estimatable">$2,530</td>,
 <td class="a-text-right mojo-field-type-money mojo-estimatable">$44,008,406</td>,
 <td class="a-text-right mojo-field-type-positive_integer">3</td>,
 <td class="a-text-left mojo-field-type-release_stud

In [41]:
# Para realizar a leitura do conteúdo que está dentro da tag podemos usar o método .get_text(), vamos utilizar o loop for para iterar por todos os dados imersos em tags <td>
for i in soup.findAll('td')[0:50]: #[0:50] faz com que seja executado apenas os primeiros caracteres.
  print(i.get_text())


1
1
Ghostbusters: Afterlife
$10,918,188
-33.6%
-
4,315
$2,530
$44,008,406
3
Sony Pictures Entertainment (SPE)


false
false
2
2
Eternals
$3,255,389
-32.1%
-54.4%
4,055
$802
$136,077,018
17
Walt Disney Studios Motion Pictures


false
false
3
3
Clifford the Big Red Dog
$2,405,161
-34.8%
-49.6%
3,628
$662
$33,534,612
12
Paramount Pictures


false
false
4
4
King Richard
$1,330,385
-37.7%
-
3,302
$402
$5,406,033
3
Warner Bros.




Saímos de um cenário em que a leitura de informações era impossível para uma morfologia interpretável. Apesar disto, ainda nos deparamos com informações que parecem estranhas, como 'false' e espaços vazios. Percebemos que, na verdade, cada conjunto separado por espaços deveria ser uma linha que informa alguns dados do dia a respeito da receita do filme. 

Será necessário executar uma lógica para organizar esses dados,  e mais uma vez o Python nos ajuda a manipular os dados e eliminar dados indesejáveis. O código abaixo visa identificar os espaços vazios e agrupar dados que deveriam estar na mesma linha.

In [11]:

#Iterando e eliminando dados indesejados por meio de listas, if-else e for. Também realizamos uma quebra de linha, cada filme possui 11 colunas. 
# Portanto, neste trecho do código também nasce a nossa tabela diária.

count=0
lista = []
lista_completa =[]
for i in soup.findAll('td'):
    if (i.get_text() == '\n') or (i.get_text() =='false') or (i.get_text() == 'true'):
        count=0
        lista =[]
    else:
        lista.append(i.get_text())
        count+=1
        if count==10: #a cada ciclo de 11 iterações realizamos um corte de linha, indicando que a próxima iteração se refere a outro filme. Cada linha possui 11 colunas.
            lista_completa.append(lista)
  

In [43]:
#Aqui notamos que nos livramos do lixo presente na captura do HTML apenas utilizando Python

lista_completa[0:5] #apresentando os 5 primeiros elementos, cada elemento é uma lista, que por sua vez é uma linha.

[['1',
  '1',
  'Ghostbusters: Afterlife',
  '$10,918,188',
  '-33.6%',
  '-',
  '4,315',
  '$2,530',
  '$44,008,406',
  '3',
  'Sony Pictures Entertainment (SPE)\n\n'],
 ['2',
  '2',
  'Eternals',
  '$3,255,389',
  '-32.1%',
  '-54.4%',
  '4,055',
  '$802',
  '$136,077,018',
  '17',
  'Walt Disney Studios Motion Pictures\n\n'],
 ['3',
  '3',
  'Clifford the Big Red Dog',
  '$2,405,161',
  '-34.8%',
  '-49.6%',
  '3,628',
  '$662',
  '$33,534,612',
  '12',
  'Paramount Pictures\n\n'],
 ['4',
  '4',
  'King Richard',
  '$1,330,385',
  '-37.7%',
  '-',
  '3,302',
  '$402',
  '$5,406,033',
  '3',
  'Warner Bros.\n\n'],
 ['5',
  '5',
  'Dune',
  '$927,038',
  '-31.8%',
  '-37.1%',
  '2,467',
  '$375',
  '$98,301,904',
  '31',
  'Warner Bros.\n\n']]

## 2. Manipulação de dados com Python (Pandas)

O Pandas é a principal biblioteca Python para manipulação de dados, permite limpeza, junção, cálculos, carregamentos, integrações e nos dá uma ótima maneira de visualizar tabelas, o objeto DataFrame nos permite enxergar as tabelas com facilidade. Vamos utilizá-lo para entender melhor os dados.

<p align=center>
<img src="https://www.seekpng.com/png/full/348-3481904_images-pandas-logo-pandas-python-logo.png" width="40%"></p>


In [13]:
#Observe que uma lista de lista facilmente se torna uma tabela de ótima apresentação com apenas 01 linha de código.

pd.DataFrame(lista_completa)

,0,1,2,3,4,5,6,7,8,9,10
0,1,1,Ghostbusters: Afterlife,"$10,918,188",-33.6%,-,"4,315","$2,530","$44,008,406",3,Sony Pictures Entertainment (SPE)\n\n
1,2,2,Eternals,"$3,255,389",-32.1%,-54.4%,"4,055",$802,"$136,077,018",17,Walt Disney Studios Motion Pictures\n\n
2,3,3,Clifford the Big Red Dog,"$2,405,161",-34.8%,-49.6%,"3,628",$662,"$33,534,612",12,Paramount Pictures\n\n
3,4,4,King Richard,"$1,330,385",-37.7%,-,"3,302",$402,"$5,406,033",3,Warner Bros.\n\n
4,5,5,Dune,"$927,038",-31.8%,-37.1%,"2,467",$375,"$98,301,904",31,Warner Bros.\n\n
5,6,6,Venom: Let There Be Carnage,"$837,297",-36.2%,-18.7%,"2,538",$329,"$206,610,673",52,Sony Pictures Entertainment (SPE)\n\n
6,7,7,No Time to Die,"$781,460",-34.5%,-30.6%,"2,407",$324,"$154,748,399",45,Metro-Goldwyn-Mayer (MGM)\n\n
7,8,8,Ron's Gone Wrong,"$292,512",-34.3%,-51.7%,"1,520",$192,"$22,152,788",31,20th Century Studios\n\n
8,9,9,The French Dispatch,"$291,954",-30%,-36.8%,805,$362,"$13,324,025",31,Searchlight Pictures\n\n
9,10,10,Belfast,"$264,905",-33.8%,-39.4%,584,$453,"$3,441,435",10,Focus Features\n\n


In [14]:
#Vamos dar nome aos bois, copiamos o nome das colunas presentes no site boxoffice para o DataFrame. Agora os dados se tornaram informação.

frame = pd.DataFrame(lista_completa, columns=['TD','YD','Release','Daily','% YD','% LW','Theaters','Avg','To Date','Days','Distributor'])

In [15]:
frame

,TD,YD,Release,Daily,% YD,% LW,Theaters,Avg,To Date,Days,Distributor
0,1,1,Ghostbusters: Afterlife,"$10,918,188",-33.6%,-,"4,315","$2,530","$44,008,406",3,Sony Pictures Entertainment (SPE)\n\n
1,2,2,Eternals,"$3,255,389",-32.1%,-54.4%,"4,055",$802,"$136,077,018",17,Walt Disney Studios Motion Pictures\n\n
2,3,3,Clifford the Big Red Dog,"$2,405,161",-34.8%,-49.6%,"3,628",$662,"$33,534,612",12,Paramount Pictures\n\n
3,4,4,King Richard,"$1,330,385",-37.7%,-,"3,302",$402,"$5,406,033",3,Warner Bros.\n\n
4,5,5,Dune,"$927,038",-31.8%,-37.1%,"2,467",$375,"$98,301,904",31,Warner Bros.\n\n
5,6,6,Venom: Let There Be Carnage,"$837,297",-36.2%,-18.7%,"2,538",$329,"$206,610,673",52,Sony Pictures Entertainment (SPE)\n\n
6,7,7,No Time to Die,"$781,460",-34.5%,-30.6%,"2,407",$324,"$154,748,399",45,Metro-Goldwyn-Mayer (MGM)\n\n
7,8,8,Ron's Gone Wrong,"$292,512",-34.3%,-51.7%,"1,520",$192,"$22,152,788",31,20th Century Studios\n\n
8,9,9,The French Dispatch,"$291,954",-30%,-36.8%,805,$362,"$13,324,025",31,Searchlight Pictures\n\n
9,10,10,Belfast,"$264,905",-33.8%,-39.4%,584,$453,"$3,441,435",10,Focus Features\n\n


In [16]:
#Note, entretanto, que não resolvemos um problema na manipulação de dados com Python. Na coluna Distributor, temos os dados \n\n que são indesejáveis, vamos retirar isto com Pandas. 

frame.Distributor.replace(r'\n\n','',regex=True,inplace=True) #Substituímos \n\n por '' vazio. inplace=True indica que vamos realizar essa substituição no próprio objeto frame, sem necessidade de criar um novo.

In [17]:
frame

,TD,YD,Release,Daily,% YD,% LW,Theaters,Avg,To Date,Days,Distributor
0,1,1,Ghostbusters: Afterlife,"$10,918,188",-33.6%,-,"4,315","$2,530","$44,008,406",3,Sony Pictures Entertainment (SPE)
1,2,2,Eternals,"$3,255,389",-32.1%,-54.4%,"4,055",$802,"$136,077,018",17,Walt Disney Studios Motion Pictures
2,3,3,Clifford the Big Red Dog,"$2,405,161",-34.8%,-49.6%,"3,628",$662,"$33,534,612",12,Paramount Pictures
3,4,4,King Richard,"$1,330,385",-37.7%,-,"3,302",$402,"$5,406,033",3,Warner Bros.
4,5,5,Dune,"$927,038",-31.8%,-37.1%,"2,467",$375,"$98,301,904",31,Warner Bros.
5,6,6,Venom: Let There Be Carnage,"$837,297",-36.2%,-18.7%,"2,538",$329,"$206,610,673",52,Sony Pictures Entertainment (SPE)
6,7,7,No Time to Die,"$781,460",-34.5%,-30.6%,"2,407",$324,"$154,748,399",45,Metro-Goldwyn-Mayer (MGM)
7,8,8,Ron's Gone Wrong,"$292,512",-34.3%,-51.7%,"1,520",$192,"$22,152,788",31,20th Century Studios
8,9,9,The French Dispatch,"$291,954",-30%,-36.8%,805,$362,"$13,324,025",31,Searchlight Pictures
9,10,10,Belfast,"$264,905",-33.8%,-39.4%,584,$453,"$3,441,435",10,Focus Features


### Criação de funções próprias e manipulações com Pandas

A substituição feita a pouco não foi o último desafio, há várias outras manipulações que serão necessárias.

Mas, como supracitado, uma vez que entendemos o básico, vamos iniciar a criação das funções próprias que serão utilizadas para iterar nas tabelas diárias. Enquanto isto, vamos superando as dificuldades a cada novo desafio.

In [18]:
# Função realiza o que já vimos, lê a página a HTML. O except entra em caso de falha, por exemplo, retorno de código 404 de página não encontrada. 
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()

    except:
        pass

In [48]:
# Com a utilização desta função podemos colocar a url de qualquer página e teremos como resposta a captura do texto html.

ConsultaWebB("https://www.boxofficemojo.com/date/2021-11-22/")[0:1000]

b'<!doctype html><html class="a-no-js" data-19ax5a9jf="dingo"><head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);\n\n\n    var ue_err_chan = \'jserr-rw\';\n(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;

In [20]:
# A função abaixo está encadeada com a função anterior. Tem como objetivo aplicar o beautiful soup ao resultado anterior. Para isto também utiliza o endereço url.

def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')    
    return soup

In [50]:
captura_html_pagina("https://www.boxofficemojo.com/date/2021-11-22/"); #retirar supressão ';' caso queira observar a saída.

In [22]:
# A função catch_frame encadeia as duas anteriores, observe que já vimos o mesmo loop for anteriormente! A função vai mais além, com apenas a url nos devolve os dados muito mais organizados em formato DataFrame do Pandas. Um ótimo progresso!!

def catch_frame (url):
    
    agente = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    headers = {'User-Agent': agente}

    response =ConsultaWebB(url)  #consulta disponibilidade da página. Passa se houver erro ou indisponibilidade
    soup = captura_html_pagina(url) #realiza leitura dos elementos da página
    
    # Organizando esqueleto da captura em tabela.
    count=0
    lista = []
    lista_completa =[]
    for i in soup.findAll('td'):
        if (i.get_text() == '\n') or (i.get_text() =='false') or (i.get_text() == 'true'):
            count=0
            lista =[]
        else:
            lista.append(i.get_text())
            count+=1
            if count==10:
                lista_completa.append(lista)
    frame = pd.DataFrame(lista_completa, columns=['TD','YD','Release','Daily','% YD','% LW','Theaters','Avg','To Date','Days','Distributor'])
    frame.Distributor.replace(r'\n\n','',regex=True,inplace=True)
    return frame

In [23]:
#Hora de testar a função com a página do dia 22 de janeiro de 2021:

catch_frame('https://www.boxofficemojo.com/date/2021-01-22/')

,TD,YD,Release,Daily,% YD,% LW,Theaters,Avg,To Date,Days,Distributor
0,1,1,The Marksman,"$597,574",+278.4%,-43.1%,"2,018",$296,"$4,661,368",8,Open Road Films (II)
1,2,3,Wonder Woman 1984,"$406,561",-34.9%,+6.1%,"2,013",$201,"$36,449,541",38,Warner Bros.
2,3,1,The Croods: A New Age,"$391,800",+240.7%,+421.5%,"1,876",$208,"$40,407,255",85,Universal Pictures
3,4,6,News of the World,"$248,410",-0.9%,+69.6%,"1,953",$127,"$9,090,275",38,Universal Pictures
4,5,5,Monster Hunter,"$208,055",+519.8%,+258%,"1,661",$125,"$9,582,881",47,Screen Gems
5,6,7,Promising Young Woman,"$125,060",+4.3%,+37.4%,"1,333",$93,"$3,704,525",38,Focus Features
6,7,8,Fatale,"$112,433",+662.7%,+259.2%,"1,175",$95,"$4,971,035",47,Lionsgate
7,8,-,Our Friend,"$85,010",-,-,543,$156,"$85,010",1,Gravitas Ventures
8,9,12,The War with Grandpa,"$35,056",+300%,+230.3%,507,$69,"$19,340,470",140,101 Studios
9,10,14,Pinocchio,"$29,727",-10.8%,-22.6%,602,$49,"$1,434,605",38,Roadside Attractions


In [24]:
frame_debbug = catch_frame('https://www.boxofficemojo.com/date/2021-01-22/')

Note que na coluna Theaters há alguns traços '-', vamos substituí-los com o intuito de viabilizar possíveis operações matemáticas futuras. 

Por exemplo, com este traço não podemos tratar a coluna Theaters como número int ou float, somente como category, string ou objeto, e isto pode representar um problema no futuro. Portanto, é um indicativo de que precisamos ganhar experiência com o tipo de dado que cada tabela diária trará para realizar as devidas limpezas e manipulações. 

In [25]:
frame_debbug['Theaters'] = frame_debbug['Theaters'].str.replace('-','0') #substituir '-' por 0 para posteriormente substitutir o tipo da coluna. 

In [26]:
frame_debbug #observe com onde havia '-' agora há 0


,TD,YD,Release,Daily,% YD,% LW,Theaters,Avg,To Date,Days,Distributor
0,1,1,The Marksman,"$597,574",+278.4%,-43.1%,"2,018",$296,"$4,661,368",8,Open Road Films (II)
1,2,3,Wonder Woman 1984,"$406,561",-34.9%,+6.1%,"2,013",$201,"$36,449,541",38,Warner Bros.
2,3,1,The Croods: A New Age,"$391,800",+240.7%,+421.5%,"1,876",$208,"$40,407,255",85,Universal Pictures
3,4,6,News of the World,"$248,410",-0.9%,+69.6%,"1,953",$127,"$9,090,275",38,Universal Pictures
4,5,5,Monster Hunter,"$208,055",+519.8%,+258%,"1,661",$125,"$9,582,881",47,Screen Gems
5,6,7,Promising Young Woman,"$125,060",+4.3%,+37.4%,"1,333",$93,"$3,704,525",38,Focus Features
6,7,8,Fatale,"$112,433",+662.7%,+259.2%,"1,175",$95,"$4,971,035",47,Lionsgate
7,8,-,Our Friend,"$85,010",-,-,543,$156,"$85,010",1,Gravitas Ventures
8,9,12,The War with Grandpa,"$35,056",+300%,+230.3%,507,$69,"$19,340,470",140,101 Studios
9,10,14,Pinocchio,"$29,727",-10.8%,-22.6%,602,$49,"$1,434,605",38,Roadside Attractions


É claro que não gostaríamos de exportar cada tabela diária tendo que digitar urls diferentes. Felizmente (e propositalmente) o projeto possui urls fáceis de automatizar. Caso fosse diferente, provavelmente precisariamos de uma biblioteca python como Selenium, em que seria possível simular cliques e preencher campos. 

Observe abaixo que a url nada mais é do que uma string, podemos concatenar quaisquer variáveis escolhidas por nós em meio ao endereço url completo. Isto será muito útil mais tarde, pois queremos iterar pelos dias para puxar várias urls, uma para cada dia-mês-ano, e enviar os dados para um banco de dados.

In [27]:
# Encarando trechos da url como variáveis:

ano = 2021
mes = 1
dia = 22

url = 'https://www.boxofficemojo.com/date/'+str(ano)+'-'+str(mes)+'-'+str(dia)
    
# Utilizando a url que criamos com as variáveis criadas. Para isto também utilizamos a função que criamos anteriormente.:

frame = catch_frame(url) 

In [28]:
# Dessa forma obtemos os dados normalmente.
frame 

,TD,YD,Release,Daily,% YD,% LW,Theaters,Avg,To Date,Days,Distributor
0,1,1,The Marksman,"$597,574",+278.4%,-43.1%,"2,018",$296,"$4,661,368",8,Open Road Films (II)
1,2,3,Wonder Woman 1984,"$406,561",-34.9%,+6.1%,"2,013",$201,"$36,449,541",38,Warner Bros.
2,3,1,The Croods: A New Age,"$391,800",+240.7%,+421.5%,"1,876",$208,"$40,407,255",85,Universal Pictures
3,4,6,News of the World,"$248,410",-0.9%,+69.6%,"1,953",$127,"$9,090,275",38,Universal Pictures
4,5,5,Monster Hunter,"$208,055",+519.8%,+258%,"1,661",$125,"$9,582,881",47,Screen Gems
5,6,7,Promising Young Woman,"$125,060",+4.3%,+37.4%,"1,333",$93,"$3,704,525",38,Focus Features
6,7,8,Fatale,"$112,433",+662.7%,+259.2%,"1,175",$95,"$4,971,035",47,Lionsgate
7,8,-,Our Friend,"$85,010",-,-,543,$156,"$85,010",1,Gravitas Ventures
8,9,12,The War with Grandpa,"$35,056",+300%,+230.3%,507,$69,"$19,340,470",140,101 Studios
9,10,14,Pinocchio,"$29,727",-10.8%,-22.6%,602,$49,"$1,434,605",38,Roadside Attractions


In [29]:
# Vamos comentar este trecho abaixo mas o mesmo estará em uma função mais a frente, 
#pois queremos esses tratamentos de dados para toda e qualquer tabela diária.

#Cria coluna Date para anotar ano, mês e dia (são as mesmas variáveis que estão na url). 
#Esta coluna anota informações sobre a data das informações extraidas no web scraping
frame['Date'] = date(ano,mes,dia)  

# Início da limpeza e preparação dos dados. 

# Na ideação do projeto não gostariamos de utilizar informações sobre variações de um dia para outro. 
#Por isso nos livramos das colunas 'TD', 'YD' e etc.
frame.drop(columns=['TD','YD','% YD','% LW'],inplace=True)
frame['Release'] = frame['Release'].astype('category')


#Substituimos quaisquer caracteres que possam aparecer nas respectivas colunas, impedindo a devida mudança de tipo de dado.
frame['Daily'] = frame['Daily'].str.replace('$','')
frame['Daily'] = frame['Daily'].str.replace(',','')
frame['Daily'] = frame['Daily'].astype('float') # Coluna daily será float (decimal)

frame['Theaters'] = frame['Theaters'].str.replace(',','')
frame['Theaters'] = frame['Theaters'].str.replace('-','0')
frame['Theaters'] = frame['Theaters'].astype('int') #Coluna Theaters será int (inteiro), teoricamente não existe 2,5 salas de cinema passando o filme x, por exemplo

frame['Avg'] = frame['Avg'].str.replace('$','')
frame['Avg'] = frame['Avg'].str.replace(',','')
frame['Avg'] = frame['Avg'].str.replace('-','0')
frame['Avg'] = frame['Avg'].astype('float') # Média de preço pode ser float (decimal)

frame['To Date'] = frame['To Date'].str.replace('$','')
frame['To Date'] = frame['To Date'].str.replace(',','')
frame['To Date'] = frame['To Date'].str.replace('-','0')
frame['To Date'] = frame['To Date'].astype('float') # Acumulado de receita pode ser float (decimal)


In [30]:
# Distributor (distribuidora do filme) é um texto, e colocamos como category. 
frame['Distributor'] = frame['Distributor'].astype('category')

# Não há tipo melhor para definir datas como o tipo Datetime. 
# Esses objetos carregam internamente diversos métodos de manipular datas, como somar dias e lidar com calendário, certamente iremos precisar de tais recursos.
frame['Date'] = pd.to_datetime(frame['Date'])

In [31]:
frame

,Release,Daily,Theaters,Avg,To Date,Days,Distributor,Date
0,The Marksman,597574.0,2018,296.0,4661368.0,8,Open Road Films (II),2021-01-22
1,Wonder Woman 1984,406561.0,2013,201.0,36449541.0,38,Warner Bros.,2021-01-22
2,The Croods: A New Age,391800.0,1876,208.0,40407255.0,85,Universal Pictures,2021-01-22
3,News of the World,248410.0,1953,127.0,9090275.0,38,Universal Pictures,2021-01-22
4,Monster Hunter,208055.0,1661,125.0,9582881.0,47,Screen Gems,2021-01-22
5,Promising Young Woman,125060.0,1333,93.0,3704525.0,38,Focus Features,2021-01-22
6,Fatale,112433.0,1175,95.0,4971035.0,47,Lionsgate,2021-01-22
7,Our Friend,85010.0,543,156.0,85010.0,1,Gravitas Ventures,2021-01-22
8,The War with Grandpa,35056.0,507,69.0,19340470.0,140,101 Studios,2021-01-22
9,Pinocchio,29727.0,602,49.0,1434605.0,38,Roadside Attractions,2021-01-22


In [32]:
#Finalmente condensamos tudo que vimos até o momento em uma função chamada busca_loop(), perceba que temos como parâmetro ano, mês e dia.
# Dessa forma conseguimos resgatar qualquer página digitando apenas as datas, internamente iremos construir a url. 


def busca_loop(ano,mes,dia):
    url = 'https://www.boxofficemojo.com/date/'+str(ano)+'-'+str(mes)+'-'+str(dia)
    
    frame = catch_frame(url) #captura html e trata com beautiful soup, improtando info entre tags td.
    
    if frame.shape[0] == 0:
        frame.loc[0] = [1,2,'Release Blank','0','0','0','0','0','0','0','Distributor Blank'] 
        #adiciona linha em branco em casos de tabela vazia
    
    frame['Date'] = date(ano,mes,dia)  #anotando data da busca,ou seja, adicionando coluna.
    
    #inicio da limpeza e preparação dos dados. 
    frame.drop(columns=['TD','YD','% YD','% LW'],inplace=True)
    frame['Release'] = frame['Release'].astype('category')

    frame['Daily'] = frame['Daily'].str.replace('$','')
    frame['Daily'] = frame['Daily'].str.replace(',','')
    frame['Daily'] = frame['Daily'].astype('float')

    frame['Theaters'] = frame['Theaters'].str.replace(',','')
    frame['Theaters'] = frame['Theaters'].str.replace('-','0')
    frame['Theaters'] = frame['Theaters'].astype('int')

    frame['Avg'] = frame['Avg'].str.replace('$','')
    frame['Avg'] = frame['Avg'].str.replace(',','')
    frame['Avg'] = frame['Avg'].str.replace('-','0')
    frame['Avg'] = frame['Avg'].astype('float')

    frame['To Date'] = frame['To Date'].str.replace('$','')
    frame['To Date'] = frame['To Date'].str.replace(',','')
    frame['To Date'] = frame['To Date'].str.replace('-','0')
    frame['To Date'] = frame['To Date'].astype('float')

    frame['Days'] = frame['Days'].str.replace(',','') #há casos de filmes que passaram 1000 dias ou mais 
    #nesse caso retiramos o separador de milhares ,
    frame['Days'] = frame['Days'].astype('int')

    frame['Distributor'] = frame['Distributor'].astype('category')

    frame['Date'] = pd.to_datetime(frame['Date'])

    return frame

In [33]:

# De forma proposital, buscamos uma data em que há página (código HTML 200) mas não há dados. Isso responde as dúvidas sobre o condicional IF dentro da função.
# Caso a quantidade de linhas da tabela buscada seja 0, o python irá criar uma tabela com apenas uma linha, apenas para anotar a data correspondente. Note que Date possui uma linha com 2020-03-29 em sua coluna.
# O motivo disto irá ficar mais claro quando formos trabalhar com o banco de dados, este truque resolve um problema que praticamente só pode ser notado na experimentando, quando incrementamos o banco dia após dia.
# Ps: Lembre-se que em março de 2020 estávamos dentro do pico da pandemia e os cinemas estavam fechados. 

busca_loop(2020,3,29) 

,Release,Daily,Theaters,Avg,To Date,Days,Distributor,Date
0,Release Blank,0.0,0,0.0,0.0,0,Distributor Blank,2020-03-29


In [34]:
busca_loop(2020,3,28) #busca a tabela em 28 de março de 2020.

,Release,Daily,Theaters,Avg,To Date,Days,Distributor,Date
0,Release Blank,0.0,0,0.0,0.0,0,Distributor Blank,2020-03-28


## 3. Criando banco de dados no SQL Server

O **pyodbc** é um módulo de código aberto desenvolvido pelas comunidades python e microsoft para facilitar o acesso ao gerenciador de banco de dados SQL Server. Queremos nos aproveitar desta comunicação para enviar os dados tratados para o SQL Server, onde iremos guardar os dados diários e podemos incrementar com novos dados diariamente rodando o código python que criamos. A partir disto, podemos puxar os dados a partir do Power BI para tratar um histórico de informações de meses ou anos.

Para este projeto foi utilizado o SQL Server 18. Trabalhou-se com um servidor de banco de dados simulado pelo Windows.

O **pyodbc** permite a utilização de consultas SQL diretamente do Python. Basicamente estabelecemos uma conexão via Python e acionamos um cursor que varre os dados da tabela e os copia para o banco de dados relacional. Operações como consulta de certas tabelas, ordenação, determinação de máximo ou mínimo de cada coluna e até mesmo eliminação do banco podem ser feitas diretamente do Python.

Neste projeto, por exemplo, vamos utilizar a função MAX() do SQL para determinar qual a última data presente no banco de dados. Isto porque precisamos incrementar justamente a próxima data. 

Vamos realizar uma consulta dentro do SQL Server e comparar com uma consulta utilizando pyodbc. Queremos saber quais foram os últimos dados inseridos no banco até a data em que escrevo este artigo, 29 de Dezembro de 2021, ordenado pela data de forma decrescente.
Em geral o comando SQL é dado como abaixo.



<p align=center>
<img src="https://www.freecodecamp.org/news/content/images/2020/08/Untitled-design-1-.png" width="40%"></p>

Como o banco de dados que vamos criar é apenas uma simulação do servidor criado pelo meu próprio windows, os outputs não serão mais visíveis. Portanto, sugiro visitar este [link](https://medium.com/@baia-science/do-python-e-sql-ao-power-bi-webscraping-do-imdb-eb727c2dd0d8) do meu Medium para acompanhar so resultados finais.

<p align=center>
<img src="https://cdn.freebiesupply.com/images/large/2x/medium-icon-white-on-black.png" width="25%"></p>



In [35]:
pip install pyodbc==4.0.30 #a versão utilizada para o projeto foi a 4.0.30, desta forma garantimos compatibilidade com as demais bibliotecas.

In [36]:
import pyodbc

In [37]:
pyodbc.version

'4.0.30'

In [38]:
#Estabelecendo conexão com o banco de dados, UID é o usuário do banco de dados, PWD é a senha. 
#Neste projeto criei um servidor próprio chamado DESKTOP-3GPDTFK\MSSQLSERVER2. O banco de dados se chama Imdb.Estas definições são desenvolvidas na criação do banco de dados. 
#Não há segredo, apenas devemos copiar para o ambiente python. 

conn = pyodbc.connect('Trusted_Connection=yes', 
driver = '{ODBC Driver 17 for SQL Server}',
server = '', 
database = '',
UID='',
PWD='')

# query é um consulta, aqui estamos digitando em linguagem SQL que gostariamos de selecionar todas as colunas do banco de dados imdb que criamos. 
# É preciso configurar o banco para que tenhamos algum resultado. 
query = '''
    select 
        * 
    from imdb
'''
sql_query = pd.read_sql_query(query,conn) #chama A função pandas read_sql_query para executar a query (pode mudar) utilizando a conexão conn. A variável sql_query guarda o retorno dado pelo SQL Sever
sql_query

Error: ignored

In [ ]:
# Para evitar surpresas com colunas que não gostariamos de incrementar ao banco de dados.
# Definimos desde já as colunas que possuem correspondência no banco de dados.
# Ou seja, não adianta termos colunas no ambiente python e que não estejam no banco de dados que criamos.

base = pd.DataFrame(columns=['Release',
 'Daily',
 'Theaters',
 'Avg',
 'To Date',
 'Days',
 'Distributor',
 'Date'])

In [ ]:

# A Função inserir dados utiliza o método cursor para inserir as tabelas que tratamos. 
# bserve que o PRINCIPAL e ÚNICO parâmetro é um objeto DateFrame, exatamente uma tabela diária qualquer já manipulada e limpa.

def InserirDados(frame):

    base = pd.DataFrame(columns=['Release',
    'Daily',
    'Theaters',
    'Avg',
    'To Date',
    'Days',
    'Distributor',
    'Date'])

    #frame_dev = base.append(frame_dev).fillna('')

    conn = pyodbc.connect('Trusted_Connection=yes', 
    driver = '{ODBC Driver 17 for SQL Server}',
    server = '', 
    database = '',
    UID='',
    PWD='')

    cursor = conn.cursor()

    for index, row in frame.iterrows():

        cursor.execute('''

            INSERT INTO imdb (
                Release,
                Daily,
                Theaters,
                Avg_,
                To_date,
                Days_,
                Distributor,
                Date_
            ) 
            values(?,?,?,?,?,?,?,?)''', 
            # o número de ? deve estar em mesma quantidade que a quantidade de variáveis. A ordem importa, trata-se de um depara. 
            # ? indica que se trata de uma variável

            row['Release'], 
            row['Daily'], 
            row['Theaters'], 
            row['Avg'],
            row['To Date'], 
            row['Days'], 
            row['Distributor'], 
            row['Date']


        )

    conn.commit()
    cursor.close()

In [ ]:
#Consulta dados no banco de dados, sendo a query qualquer comando sql, não somente SELECT * FROM Imdb como citado anteriormente.
def SQLConsulta(query):
    conn = pyodbc.connect('Trusted_Connection=yes', 
        driver = '{ODBC Driver 17 for SQL Server}',
        server = '', 
        database = '',
        UID='',
        PWD='')



    sql_query = pd.read_sql_query(query,conn)
    return sql_query
    

In [ ]:
#Função para truncate simplesmente apaga os dados do banco de dados NomeTabela. Deve ser utilizado com muito cuidado. 

def SQLTruncate(NomeTabela):

    conn = pyodbc.connect('Trusted_Connection=yes', 
        driver = '{ODBC Driver 17 for SQL Server}',
        server = '', 
        database = '',
        UID='',
        PWD='')

    cursor = conn.cursor()

    cursor.execute(f'''

                   TRUNCATE TABLE {NomeTabela}

                   ''')

    conn.commit()
    cursor.close()

In [ ]:
SQLConsulta('SELECT * FROM dbo.imdb') # similar (não igual) a SELECT * FROM imdb

In [ ]:
#SQLTruncate('dbo.imdb') #No código original deixei esta linha comentada para evitar apagar o banco de dados. Puxar os dados de 1 ano inteiro pode levar até 40 minutos devido ao delay entre aquisições que foi adicionado e o processamento inerente. 
#SQLConsulta('SELECT * FROM dbo.imdb')

In [ ]:
date.today() #puxa informações sobre a data de hoje em formato Datetime

## 4. Atualização Incremental

A próxima etapa é bastante importante. Se trata da nossa atualização incremental. O objetivo deste trecho é buscar páginas que ainda não foram adicionadas ao banco de dados. Isto nos fará ganhar bastante tempo, pois não precisaremos executar horas de web scraping para obter sempre os mesmos dados toda vez que formos realizar uma análise.

Mas como conseguir isto? Basicamente utilizamos a função MAX() na query SQL para buscar a última data presente no banco de dados. A partir disto, importamos a informação de data para o pandas, somamos 01 dia e utilizamos uma função Python de captura de tabela para resgatar a tabela seguinte. 

Mas e se o banco de dados estiver vazio, qual a referência? Nesse caso utilizamos uma data de kick-off, que no nosso caso, é o dia 01 de Janeiro de 2020, mas poderia ser qualquer data após 1984 (quando a série histórica do IMDB começa). A execução de 2020 e 2021 completos durou em torno de 80 minutos. Puxar os dados de 1984 até hoje seria ainda mais custoso operacionalmente. Porém, uma vez executado, os dados estariam guardados e só precisaríamos utilizar a atualização incremental para adicionar somente as datas mais recentes. Inclusive, poderíamos exportar os dados em formato backup compactado .bak caso haja necessidade.

Para finalizarmos nossa análise superficial do que foi feito em Python, criamos uma função que recebe a informação de quantos dias queremos incrementar no banco de dados de forma incremental. Se digitarmos que queremos incrementar 30 tabelas e a última linha presente no banco de dados é de 31 de Agosto, iremos incrementar todo o mês de Setembro automaticamente. Caso o banco esteja vazio, portanto, não havendo referência, podemos determinar uma data início como supracitado.

<br>


<p align=center>
<img src="https://github.com/baiascience/imdb/blob/main/loop.png?raw=true" width="70%"></p>

In [ ]:
def IncrementarDados ():
    dados_ano = SQLConsulta(f'select Date_ = max(Date_) from dbo.imdb')
    ultima_data = dados_ano.loc[0,'Date_']
    if ultima_data==None:
        proxima_data=date(2020,1,1) #Se o banco estiver vazio, iniciar por 01 de Janeiro de 2020.
        print('Eita, o banco está vazio. Mas não se preocupe vou iniciar por ',proxima_data)
        print('---------------- \n')
    elif date.today() < ultima_data+timedelta(days=1):   #Se o banco não estiver vazio mas superamos a data de hoje: "escreve erro de dados indisponíveis. Data futura"
        print('Dados Indisponíveis. Data futura.')
        print('---------------- \n')

    else:
        print('Ultima data cadastrada no banco: ',ultima_data) #Se o banco não estiver vazio e ainda não chegamos ao dia de hoje: utiliza a última data presente no banco e soma 1 dia utilizando a função timedelta (uma das facilidades de trabalhar com os objetos datetime)
        proxima_data =  ultima_data + timedelta(days=1)
        print('Próxima data a ser considerada: ',proxima_data)
        
        print('Dia: ',proxima_data.day,'Mês: ',proxima_data.month,'Ano: ',proxima_data.year)
        print('---------------- \n')
        
        frame_busca = busca_loop(proxima_data.year,proxima_data.month,proxima_data.day) #chama a função busca_loop() para capturar a tabela. 
        
    return InserirDados(frame_busca) #após capturar a tabela, insere a tabela, utilizando a biblioteca pyodbc.


In [ ]:
#InserirDados(busca_loop(2021,1,22)) forçar input



In [ ]:
SQLConsulta(f'SELECT * FROM dbo.imdb') #lendo após inserir novo frame

A última função Python é a função que irá incrementar as tabelas uma **quantidade** de vezes no Banco de dados. Iternamente, irá executar a função IncrementarDados() até finalizar a **quantidade** de vezes, o loop não será encerrando antes disso. 

In [ ]:
def LoopIncremento (quantidade):
    erro=0
    for i in range(quantidade):
        try:
            print('Iniciando iteração #',i)
            IncrementarDados()
        except:
            print('Busca não encontrada')
            IncrementarDados() 
            erro=erro+1
            pass
    print(erro)
    

In [ ]:
quantidade = int(input('Defina a quantidade de incrementos em dias: \n')) #digita quantidade de vezes

In [ ]:
LoopIncremento(quantidade) #aciona o incremento em loop
    

In [ ]:
input_manual = input('Deseja inserir dados de algum dia manualmente?') #pergunta antes de inserir qualquer tabela manualmente.

In [ ]:
InserirDados(busca_loop(2021,12,23)) #função extra para inserir uma tabela "manualmente"

<p align=center>
<img src="https://cdn.freebiesupply.com/images/large/2x/medium-icon-white-on-black.png" width="25%"></p>


#Confira o Medium para ver o resultado final: [Ir para o Medium](https://medium.com/@baia-science/do-python-e-sql-ao-power-bi-webscraping-do-imdb-eb727c2dd0d8)
<br>

<p align=center>
<img src="https://symbols-electrical.getvecta.com/stencil_92/28_power-bi.e2406d2fee.png" width="25%"></p>




#O dashboard pode ser acesso a partir deste link: [Ir para o Dashboard](https://app.powerbi.com/view?r=eyJrIjoiYjFlOGY3ZjMtYTg1MC00OWI3LWEzMjgtNTJjMWM3YTdhNjE5IiwidCI6ImUyMDcxNzllLTExMzQtNGMyYi05ZTM1LWRhMmJmN2NlOWQ5MyJ9)

<p align=center>
<img src="https://logosmarcas.net/wp-content/uploads/2020/04/Linkedin-Logo.png" width="20%"></p>


#Contato: [Ir para o Linkedin](https://www.linkedin.com/in/matheus-baia-5872b662/)



---

<p align=center>
<img src="https://github.com/baiascience/baia_science_main/blob/main/baia-science_black.png?raw=true" width="20%"></p>
by: Matheus Baia

---


